## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `rerank_rag.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [94]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Carregando arquivo

Vamos importar as libs e definir a chave da API da OpenAI. Em seguida, importaremos os modelos de embeddings e chat da OpenAI. O modelo de embeddings será o "text-embedding-3-small" e o modelo de chat será o "gpt-3.5-turbo". Limitaremos a quantidade de tokens de saída com o parâmetro "max-tokens". Carregaremos um PDF da pasta root e usaremos o PyPDF para separar as páginas do documento.

In [95]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [96]:
# Carregar modelos Open AI - Embedding e chat

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=500,
)

In [97]:
# Carregar o PDF
pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [ ]:
# total de paginas do arquivo .pdf
len(pages)

31

### Separando os Chunks

[Documentação sobre ajuste no banco de dados](https://docs-rocketseat.notion.site/Ajuste-aula-de-Rerank-193395da5770808aba49c3f49914ec2c)

Vamos separar o PDF em chunks usando o text splitter com chunk size de 4000 e overlap de 20. Criamos a função de splitter e aplicamos o text_splitter.split() nas páginas do documento. Em seguida, salvamos os chunks no vectorDB, utilizando o embedding model e o diretório NaiveDB.

In [ ]:
# Separar em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [ ]:
# Salvar os Chunks no VectorDB
vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="naiveDB")

### Método de Rerank retriever

Neste trecho da aula, foi abordado o processo de carregamento do banco de dados e a criação do NaiveRetriever, que é a primeira etapa do Contextual Compression RAG. O instrutor explicou a importância de buscar mais documentos para permitir um ranqueamento mais preciso. Em seguida, demonstrou como acessar o Cohere Rerank no Google e criar uma API Key gratuita para utilização. Por fim, mostrou como construir o Compression Retriever, que utiliza o Base Compressor e o Base Retriever para otimizar o ranqueamento dos documentos.

In [ ]:
# Carregar o DB
naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [102]:
rerank = CohereRerank(model="rerank-v3.5", top_n=3)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever,
)


### Construindo o Prompt e Chain

Neste resumo, foi abordado o processo de criação de um template para responder perguntas sobre legislação e tecnologia. Foi mencionado o uso de um prompt, a configuração da recuperação de informações e a criação de uma cadeia de processamento. Houve um problema com a coleta de documentos devido a uma alteração no nome de um banco de dados, que foi corrigido. Ao final, foi mostrado o resultado da resposta utilizando o método do Contextual Retriever.

In [ ]:
# especificar o prompt com pergunta e contexto
TEMPLATE = """
    Você é um especialista em legislação e tecnologia. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [31]:
setup_retrival = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})
output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrival | rag_prompt | llm | output_parser

### Response e conclusão

Neste resumo, foi abordado o processo de criação de um sistema de busca e ranqueamento de documentos. Foram mencionadas etapas como a criação do banco de dados, importação de modelos, separação em chunks, criação do Reranker e do Compressor Retriever. Também foi destacada a utilização de um modelo externo para o reranqueamento. Na próxima aula, será abordado o deploy de um modelo de RAG como serviço de API para integração em aplicações.

In [32]:
compressor_retrieval_chain.invoke("Quais os principais riscos do marco legal de ia")

'Como não há um contexto específico fornecido, posso destacar alguns dos principais riscos do marco legal de IA (Inteligência Artificial) que geralmente são discutidos:\n\n1. Violação da privacidade: O uso de IA pode envolver a coleta e processamento de grandes quantidades de dados pessoais, o que pode resultar em violações de privacidade se não houver regulamentações adequadas para proteger essas informações.\n\n2. Discriminação algorítmica: Algoritmos de IA podem reproduzir e amplificar preconceitos existentes, resultando em decisões discriminatórias em áreas como contratação, empréstimos, justiça criminal, entre outros.\n\n3. Responsabilidade e transparência: A complexidade dos sistemas de IA pode dificultar a atribuição de responsabilidade em caso de erros ou danos causados \u200b\u200bpor decisões automatizadas. Além disso, a falta de transparência nos algoritmos pode dificultar a compreensão de como as decisões são tomadas.\n\n4. Segurança cibernética: A integração de sistemas de